# Build a Sales Bot with Weaviate and Llama 3 on OctoAI

## Overview

In this notebook you'll take an Amazon product reviews dataset from Kaggle and use Llama 3 to obtain product review summaries, upsert those summaries in a vector database, then use Retrieval Augmented Generation (RAG) to power a sales chatbot that can make targeted product recommendations.

Let's take a look at the overall workflow:
1. We start with a dataset that contains over 10,000 reviews across 900 Amazon musical instruments and accessories.
2. Using Llama 3 70B (hosted on OctoAI), we generate summaries of product reviews for each product from the 20 most recent reviews. We format the summaries in JSON format.
3. We then take the summaries and upsert them into a vector database (Weaviate in this case)
4. We then use this vector database and Llama 3 70B instruct (hosted on OctoAI) to build a RAG-based sales chatbot that provides targeted recommendations to the user based on the products that are present in the inventory.

### OctoAI
We'll use [OctoAI](https://octo.ai/) to power all of the GenAI model needs of this notebook: LLMs, image gen, image animation.
* To use OctoAI, you'll need to go to https://octoai.cloud/ and sign in using your Google or GitHub account.
* Next you'll need to generate an OctoAI API token by following these [instructions](https://octo.ai/docs/getting-started/how-to-create-an-octoai-access-token). Keep the API token in hand, we'll need it further down in this notebook.

In this example we will use the Llama 3 70b instruct model. You can find more on Llama models on the [OctoAI text generation solution page](https://octoai.cloud/text).

### Weaviate
We'll use Weaviate Cloud (WCD) as our vector database. You can create an account and a free 14-day free cluster [here](https://console.weaviate.cloud/). 

You'll need to obtain the Endpoint URL and API key to use the cluster for this demo.

### Local Python Notebook
We highly recommend launching this notebook from a fresh python environment, for instance you can run the following:
```
python3 -m venv .venv         
source .venv/bin/activate
```
All you need to run this notebook is to install jupyter notebook with `python3 -m pip install notebook` then run `jupyter notebook` ([link](https://jupyter.org/install)) in the same directory as this `.ipynb` file.

In [ ]:
# Let's start by installing the appropriate python packages
! pip install openai weaviate-client pandas gradio pydantic

## Part 1: Review Summarization

Let's start by importing all of the packages we need for this example

In [ ]:
import gradio
import json
import os
import openai
import weaviate
from getpass import getpass
from json import loads
from pandas import DataFrame, concat, read_csv
from pydantic import BaseModel, Field
from typing import List

Enter your OctoAI, Weaviate, and OpenAI tokens below

In [ ]:
# Get OctoAI API token for Llama 2 & 3
OCTOAI_API_TOKEN = getpass()
os.environ["OCTOAI_API_TOKEN"] = OCTOAI_API_TOKEN

In [ ]:
# Get WCS API key
WCS_API_KEY = getpass()
os.environ["WCS_API_KEY"] = WCS_API_KEY

In [ ]:
# Get WCS URL (REST Endpoint)
WCS_URL = getpass()
os.environ["WCS_URL"] = WCS_URL

In [ ]:
# First let's load the dataset from Kaggle: https://www.kaggle.com/datasets/eswarchandt/amazon-music-reviews
df = read_csv('https://raw.githubusercontent.com/tmoreau89/image-assets/main/datasets/kaggle-amazon-musical-instrument-reviews/Musical_instruments_reviews.csv')

Set `product_record_limit` to a lower number if you just want to do a test run

In [ ]:
# Set a product record limit
product_record_limit = 900

In [ ]:
# List all of the unique ASIN:
asin_list = df.asin.unique()
print("There are {} unique products in the music product inventory".format(len(asin_list)))

For each one of the unique products, let's group the reviews together and sort them by how recent they are

In [ ]:
# Get the reviews for the product ASIN, sorted by recency and store in dict
review_dict = {}
for asin in asin_list[0:product_record_limit]:
    reviews = df.loc[df['asin'] == asin]\
                .sort_values(["unixReviewTime"], axis=0, ascending=False)\
                .reviewText.tolist()
    review_dict[asin] = reviews

To be able to store our summaries into our vector DB, we need to have the fields formatted into a JSON object. We use Pydantic base class model here to define the JSON schema.

In [ ]:
# Define the Pydantic model that specifies how our output should be formatted
class ProductRecord(BaseModel):
    """The record of a given product"""
    description: str = Field(description="Description of the product")
    name: str = Field(description="Name of the product")
    review_summary: str = Field(description="Summary of all of the reviews")
    ASIN: str = Field(description="ASIN of the product")
    features: str = Field(description="Features of the product based on the reviews")

We define our prompt template below.

In [ ]:
# Prepare a prompt template
template = '''
Here are product reviews for a music product with an ID of {asin}.
 - Provide:
     - the product "description",
     - the product "name",
     - a summary of all the reviews as "review_summary",
     - the "ASIN" and
     - and the product "features" based on the content of these reviews.

The reviews for the product are: {reviews}
'''

We initialize the OctoAI client using OpenAI's API. All we have to do is override the `base_url` and `api_key`.

In [ ]:
# Init OctoAI client
client = openai.OpenAI(
    base_url="https://text.octoai.run/v1",
    api_key=os.environ["OCTOAI_API_TOKEN"]
)

Iterate over all product ASINs and summarize the top 20 most recent reviews. Note: this takes a while to run unless we parallelize it.

In [ ]:
import concurrent.futures

# Max concurrent threads
MAX_THREADS = 4

# Task to run in parallel
def task(idx, asin, review_list):
    print(f'Getting review summary {idx} of {len(review_dict)}, ASIN: {asin}')
    response = client.chat.completions.create(
        model="meta-llama-3-70b-instruct",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": template.format(
                asin = asin,
                reviews = review_list[0:20] # Only get the top 20 most recent reviews
            )},
        ],
        temperature=0,
        max_tokens=4000,
        extra_body={
            "response_format": {
                "type": "json_object",
                "schema": ProductRecord.model_json_schema()
              }
        }
    )
    print("\n{}\n".format(response.choices[0].message.content))
    summary = loads(response.choices[0].message.content)
    summary["ASIN"] = asin

    return summary

# Execute task in parallel
futures = []
review_summaries = []
counter = 0
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    for asin, review_list in review_dict.items():
        # Submit tasks to the executor
        futures.append(executor.submit(task, counter, asin, review_list))
        counter += 1
    # Collect the results
    review_summaries = [future.result() for future in concurrent.futures.as_completed(futures)]

review_summaries = DataFrame(review_summaries)
print(review_summaries)

# Part 2: Retrieval Augmented Generation

For our RAG use case we're going to rely on Weaviate vector database and on an OpenAI embedding model.

When you define your collection, you'll need to provide properties, i.e. object attributes that you want to store in the collection. These properties map 1:1 to the JSON dictionary keys defined earlier for the `ProductRecord` Pydantic base model.

In [ ]:
# Connect to WCS
wcs_client = weaviate.connect_to_wcs(
    cluster_url=os.getenv("WCS_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WCS_API_KEY")),
    headers={
        "X-OctoAI-Api-Key": os.environ["OCTOAI_API_TOKEN"]
    }
)

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

# Delete collections
if wcs_client.collections.exists("Products"):
    wcs_client.collections.delete("Products")

# Create the collection for products
collection = wcs_client.collections.create(
    name="Products",
    vectorizer_config=[
        Configure.NamedVectors.text2vec_octoai(
            name="title_vector",
            source_properties=["review_summary"],
            model="thenlper/gte-large"
        )
    ],
    generative_config=Configure.Generative.octoai(
        model = "meta-llama-3-70b-instruct",
        max_tokens = 1000,
        temperature = 0.1,
        base_url = "https://text.octoai.run"
    ),
    properties=[
        Property(name="ASIN", data_type=DataType.TEXT),
        Property(name="name", data_type=DataType.TEXT),
        Property(name="review_summary", data_type=DataType.TEXT),
        Property(name="features", data_type=DataType.TEXT),
        Property(name="description", data_type=DataType.TEXT),
    ]
)
print("Collection Created!")

Now we upsert all of the vectors into the databse using OpenAI's embedding model.

In [ ]:
# Convert df to JSON string and then to a list of dictionaries
data = review_summaries.to_json(orient='records')
data_list = json.loads(data)

# Insert data into Weaviate
items_to_insert = []
with collection.batch.dynamic() as batch:
    for item in data_list:
        weaviate_obj = {
            "ASIN": item["ASIN"],
            "name": item["name"],
            "description": item["description"],  \
            "features": item["features"],
            "review_summary": item["review_summary"]
        }
        batch.add_object(
            properties=weaviate_obj
        )

# Check that import went well
if(len(collection.batch.failed_objects)>0):
    print("Import complete with errors")
    for err in collection.batch.failed_objects:
        print(err)
else:
    print("Import complete with no errors")

Let's test that vector search is working fine.

In [ ]:
# Similarity search
response = collection.query.near_text(
    query="easy to learn instrument",
    limit=3
)
for o in response.objects:
    print(o.properties)

Let's now try to run a hybrid search on the following query below.
Hybrid search combines the results of a vector search and a keyword (BM25F) search by fusing the two result sets.
It will return the 3 closest entries in the database according to the search criteria.

In [ ]:
# Hybrid search
response = collection.query.hybrid(
    query="ukulele",
    limit=3
)
for o in response.objects:
    print(o.properties)

Let's now define a helper function that gives us the relevant context given a string query. Let's see what it returns based on the question: "What is a good beginner harmonica"

Great, we're now ready to build a sales assistant helper function.

We first define a prompt template for Llama 3 - based on the context provided by the vector hybrid search (i.e. collection of product summaries of relevance to the question), provide a helpful recommendation to the customer.

Also provide links to the product that the user can click on to view the product on Amazon's website. For that we use the fact that any product referenced by its aSIN can be accessed at the following url: `https://www.amazon.com/exec/obidos/ASIN/<insert aSIN here>`

In [ ]:
template = """
You are a sales assistant. Answer the user questions as helpfully as possible with plenty of detail.

Provide a reference to each product you mention with hyperlinks:
* Provide the name of the product
* Embed the hyperlink in the name of the product as follows
    * If the product name is "Solid Electric Guitar Case with Accessories Compartment"
    * And the aSIN is "B001EL6I8W"
    * Format the reference as follows:
         [Solid Electric Guitar Case with Accessories Compartment](https://www.amazon.com/exec/obidos/ASIN/B001EL6I8W)
* Be helpful: provide insights on what the customers like or dislike about the products.
"""

def sales_assistant(question):
    response = collection.generate.near_text(
        query=question,
        grouped_task=template,
        limit=2
    )
    return response.generated

In [ ]:
# Now test it out with any query you'd like to put in!
print(sales_assistant("What is a must have pedal for my brand new guitar?"))

**Authors**
- Thierry Moreau, OctoAI - tmoreau@octo.ai
- Jonathan Tuite, Weaviate - jon@weaviate.io

**Documentation**

Learn about OctoAI's integration in Weaviate [here](https://weaviate.io/developers/weaviate/model-providers/octoai). 